# **ALL MODELS - POSTa GAN**

## **Import Dependencies**

In [1]:
# !rm -r /kaggle/working

In [2]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Number of devices: 2


In [3]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from tensorflow.keras import layers as L
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

In [4]:
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B0
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

# **Data Loader**

In [5]:
# DataLoader
train_data = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/adverserial-stsd',
    batch_size=128,
    image_size=(81, 81),
    label_mode="categorical",
    labels = "inferred",
    seed=42,
    validation_split=0.2,
    subset='training',
)

val_data = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/adverserial-stsd',
    batch_size=128,
    image_size=(81, 81),
    label_mode="categorical",
    labels = "inferred",
    seed=42,
    validation_split=0.2,
    subset='validation',
)

Found 197169 files belonging to 20 classes.
Using 157736 files for training.
Found 197169 files belonging to 20 classes.
Using 39433 files for validation.


In [6]:
NUM_CLASSES=len(os.listdir('/kaggle/input/adverserial-stsd'))

In [7]:
train_data.as_numpy_iterator().__next__()[0].shape

(128, 81, 81, 3)

In [8]:
train_data.as_numpy_iterator().__next__()[1].shape

(128, 20)

# **Model Prep**

In [9]:
# Create custom F1 Score Metrics

from keras import backend as K

def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

In [10]:
steps_per_epoch = len(train_data)
EPOCHS = 128

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=4, min_lr=0.000001)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=8,# if val loss decreases for 8 epochs in a row, stop training
                                                  restore_best_weights = True)

# **Model Creations**

In [11]:
# Model

def create_model_1(shape=(81, 81, 3)):
    base_model = EfficientNetV2B0(
        include_top = False,
        input_shape = shape,
        weights = "imagenet",
    )
    
    base_model.trainable = False
#     for layer in base_model.layers[-3:]:
# #         print(layer)
#         layer.trainable = True
        
    x = base_model.output
    x = L.Flatten(name="flatten")(x)
    x = L.Dense(NUM_CLASSES, activation="softmax", name="Output-layer")(x)
    
    model = tf.keras.models.Model(base_model.input, x, name="EfficientNetV2B0")
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.experimental.SGD(),
        metrics = ["accuracy",tf.keras.metrics.AUC(name="auc"),
                               tf.keras.metrics.Recall(name="recall"),
                               tf.keras.metrics.Precision(name="precision"),
                              f1_score]
    )
    
    return model

In [12]:
# Model

def create_model_2(shape=(81, 81, 3)):
    base_model = EfficientNetB0(
        include_top = False,
        input_shape = shape,
        weights = "imagenet",
    )
    
    base_model.trainable = False
#     for layer in base_model.layers[-3:]:
# #         print(layer)
#         layer.trainable = True
        
    x = base_model.output
    x = L.Flatten(name="flatten")(x)
    x = L.Dense(NUM_CLASSES, activation="softmax", name="Output-layer")(x)
    
    model = tf.keras.models.Model(base_model.input, x, name="EfficientNetB0")
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.experimental.Ftrl(),
        metrics = ["accuracy",tf.keras.metrics.AUC(name="auc"),
                               tf.keras.metrics.Recall(name="recall"),
                               tf.keras.metrics.Precision(name="precision"),
                              f1_score]
    )
    
    return model

In [13]:
# Model

def create_model_3(shape=(81, 81, 3)):
    base_model = InceptionV3(
        include_top = False,
        input_shape = shape,
        weights = "imagenet",
    )
    
    base_model.trainable = False
#     for layer in base_model.layers[-3:]:
# #         print(layer)
#         layer.trainable = True
        
    x = base_model.output
    x = L.Flatten(name="flatten")(x)
    x = L.Dense(NUM_CLASSES, activation="softmax", name="Output-layer")(x)
    
    model = tf.keras.models.Model(base_model.input, x, name="InceptionV3")
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.experimental.SGD(),
        metrics = ["accuracy",tf.keras.metrics.AUC(name="auc"),
                               tf.keras.metrics.Recall(name="recall"),
                               tf.keras.metrics.Precision(name="precision"),
                              f1_score]
    )
    
    return model

In [14]:
# Model

def create_model_4(shape=(81, 81, 3)):
    base_model = ResNet50(
        include_top = False,
        input_shape = shape,
        weights = "imagenet",
    )
    
    base_model.trainable = False
#     for layer in base_model.layers[-3:]:
# #         print(layer)
#         layer.trainable = True
        
    x = base_model.output
    x = L.Flatten(name="flatten")(x)
    x = L.Dense(NUM_CLASSES, activation="softmax", name="Output-layer")(x)
    
    model = tf.keras.models.Model(base_model.input, x, name="ResNet50")
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.experimental.SGD(),
        metrics = ["accuracy",tf.keras.metrics.AUC(name="auc"),
                               tf.keras.metrics.Recall(name="recall"),
                               tf.keras.metrics.Precision(name="precision"),
                              f1_score]
    )
    
    return model

In [15]:
# Model

def create_model_5(shape=(81, 81, 3)):
    base_model = Xception(
        include_top = False,
        input_shape = shape,
        weights = "imagenet",
    )
    
    base_model.trainable = False
    for layer in base_model.layers[-3:]:
#         print(layer)
        layer.trainable = True
        
    x = base_model.output
    x = L.Flatten(name="flatten")(x)
    x = L.Dense(512, activation="relu", name="Dense-1")(x)
    x = L.Dropout(0.4, name="Final-Dropout")(x)
    x = L.Dense(NUM_CLASSES, activation="softmax", name="Output-layer")(x)
    
    model = tf.keras.models.Model(base_model.input, x, name="Xception")
    model.compile(
        loss="categorical_crossentropy",
        optimizer="Adam",
        metrics = ["accuracy",tf.keras.metrics.AUC(name="auc"),
                               tf.keras.metrics.Recall(name="recall"),
                               tf.keras.metrics.Precision(name="precision"),
                              f1_score]
    )
    
    return model

In [17]:
# Model

def create_model_6(shape=(81, 81, 3)):
    base_model = MobileNetV2(
        include_top = False,
        input_shape = shape,
        weights = "imagenet",
    )
    
    base_model.trainable = False
#     for layer in base_model.layers[-3:]:
# #         print(layer)
#         layer.trainable = True
        
    x = base_model.output
    x = L.Flatten(name="flatten")(x)
    x = L.Dense(NUM_CLASSES, activation="softmax", name="Output-layer")(x)
    
    model = tf.keras.models.Model(base_model.input, x, name="MobileNetV2")
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.experimental.SGD(),
        metrics = ["accuracy",tf.keras.metrics.AUC(name="auc"),
                               tf.keras.metrics.Recall(name="recall"),
                               tf.keras.metrics.Precision(name="precision"),
                              f1_score]
    )
    
    return model

# **Train Models**

In [17]:
model_1 = create_model_1()

# Fit the model
history_1 = model_1.fit(
    train_data,
    epochs = EPOCHS,
    steps_per_epoch = steps_per_epoch,
    validation_data = val_data,
    validation_steps = len(val_data),
    callbacks = [reduce_lr],
)

model_1.save("/kaggle/working/EfficientNetV2B0.h5")
pd.DataFrame(history_1.history).to_csv("/kaggle/working/History-EfficientNetV2B0.csv")

24274472/24274472 [==============================] - 0s 0us/step
Epoch 1/128


2024-01-26 07:47:58.749422: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inEfficientNetV2B0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1233/1233 [==============================] - 155s 118ms/step - loss: 0.5049 - accuracy: 0.8663 - auc: 0.9940 - recall: 0.7590 - precision: 0.9680 - f1_score: 0.8401 - val_loss: 0.2514 - val_accuracy: 0.9384 - val_auc: 0.9987 - val_recall: 0.8887 - val_precision: 0.9788 - val_f1_score: 0.9313 - lr: 0.0100
Epoch 2/128
1233/1233 [==============================] - 65s 52ms/step - loss: 0.2384 - accuracy: 0.9421 - auc: 0.9986 - recall: 0.8976 - precision: 0.9782 - f1_score: 0.9360 - val_loss: 0.1805 - val_accuracy: 0.9550 - val_auc: 0.9992 - val_recall: 0.9243 - val_precision: 0.9825 - val_f1_score: 0.9524 - lr: 0.0100
Epoch 3/128
1233/1233 [==============================] - 60s 48ms/step - loss: 0.1864 - accuracy: 0.9538 - auc: 0.9991 - recall: 0.9225 - precision: 0.9817 - f1_score: 0.9511 - val_loss: 0.1487 - val_accuracy: 0.9630 - val_auc: 0.9994 - val_recall: 0.9391 - val_precision: 0.9848 - val_f1_score: 0.9603 - lr: 0.0100
Epoch 4/128
1233/1233 [==============================] - 60s 4

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1233/1233 [==============================] - 56s 45ms/step - loss: 0.0599 - accuracy: 0.9860 - auc: 0.9998 - recall: 0.9784 - precision: 0.9924 - f1_score: 0.9853 - val_loss: 0.0638 - val_accuracy: 0.9827 - val_auc: 0.9998 - val_recall: 0.9761 - val_precision: 0.9895 - val_f1_score: 0.9827 - lr: 0.0100
Epoch 28/128
1233/1233 [==============================] - 56s 46ms/step - loss: 0.0595 - accuracy: 0.9858 - auc: 0.9998 - recall: 0.9786 - precision: 0.9923 - f1_score: 0.9854 - val_loss: 0.0632 - val_accuracy: 0.9829 - val_auc: 0.9997 - val_recall: 0.9761 - val_precision: 0.9898 - val_f1_score: 0.9829 - lr: 0.0100
Epoch 29/128
1233/1233 [==============================] - 56s 45ms/step - loss: 0.0583 - accuracy: 0.9863 - auc: 0.9999 - recall: 0.9790 - precision: 0.9926 - f1_score: 0.9857 - val_loss: 0.0627 - val_accuracy: 0.9828 - val_auc: 0.9997 - val_recall: 0.9769 - val_precision: 0.9893 - val_f1_score: 0.9831 - lr: 0.0100
Epoch 30/128
1233/1233 [==============================] - 56s 

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
model_2 = create_model_2()

# Fit the model
history_2 = model_2.fit(
    train_data,
    epochs = EPOCHS,
    steps_per_epoch = steps_per_epoch,
    validation_data = val_data,
    validation_steps = len(val_data),
    callbacks = [reduce_lr],
)

model_2.save("/kaggle/working/EfficientNetB0.h5")
pd.DataFrame(history_2.history).to_csv("/kaggle/working/History-EfficientNetB0.csv")

16705208/16705208 [==============================] - 0s 0us/step
Epoch 1/128


2024-01-26 09:54:11.105558: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inEfficientNetB0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1233/1233 [==============================] - 77s 56ms/step - loss: 1.0834 - accuracy: 0.7262 - auc: 0.9745 - recall: 0.4415 - precision: 0.9703 - f1_score: 0.5886 - val_loss: 0.7261 - val_accuracy: 0.8185 - val_auc: 0.9893 - val_recall: 0.6216 - val_precision: 0.9731 - val_f1_score: 0.7577 - lr: 0.0010
Epoch 2/128
1233/1233 [==============================] - 66s 53ms/step - loss: 0.6883 - accuracy: 0.8270 - auc: 0.9902 - recall: 0.6450 - precision: 0.9712 - f1_score: 0.7742 - val_loss: 0.5782 - val_accuracy: 0.8556 - val_auc: 0.9933 - val_recall: 0.7072 - val_precision: 0.9742 - val_f1_score: 0.8191 - lr: 0.0010
Epoch 3/128
1233/1233 [==============================] - 66s 54ms/step - loss: 0.5824 - accuracy: 0.8544 - auc: 0.9929 - recall: 0.7071 - precision: 0.9723 - f1_score: 0.8182 - val_loss: 0.5057 - val_accuracy: 0.8755 - val_auc: 0.9948 - val_recall: 0.7498 - val_precision: 0.9752 - val_f1_score: 0.8472 - lr: 0.0010
Epoch 4/128
1233/1233 [==============================] - 69s 56m

In [19]:
model_3 = create_model_3()

# Fit the model
history_3 = model_3.fit(
    train_data,
    epochs = EPOCHS,
    steps_per_epoch = steps_per_epoch,
    validation_data = val_data,
    validation_steps = len(val_data),
    callbacks = [reduce_lr],
)

model_3.save("/kaggle/working/InceptionV3.h5")
pd.DataFrame(history_3.history).to_csv("/kaggle/working/History-InceptionV3.csv")

87910968/87910968 [==============================] - 0s 0us/step
Epoch 1/128
1233/1233 [==============================] - 65s 48ms/step - loss: 1904.3680 - accuracy: 0.3504 - auc: 0.6588 - recall: 0.3504 - precision: 0.3504 - f1_score: 0.3505 - val_loss: 1543.7593 - val_accuracy: 0.3142 - val_auc: 0.6396 - val_recall: 0.3142 - val_precision: 0.3142 - val_f1_score: 0.3139 - lr: 0.0100
Epoch 2/128
1233/1233 [==============================] - 59s 47ms/step - loss: 1012.2375 - accuracy: 0.4500 - auc: 0.7113 - recall: 0.4500 - precision: 0.4500 - f1_score: 0.4500 - val_loss: 1068.7379 - val_accuracy: 0.4593 - val_auc: 0.7163 - val_recall: 0.4593 - val_precision: 0.4593 - val_f1_score: 0.4589 - lr: 0.0100
Epoch 3/128
1233/1233 [==============================] - 57s 46ms/step - loss: 836.3301 - accuracy: 0.4847 - auc: 0.7297 - recall: 0.4847 - precision: 0.4847 - f1_score: 0.4848 - val_loss: 943.1642 - val_accuracy: 0.3693 - val_auc: 0.6686 - val_recall: 0.3693 - val_precision: 0.3693 - val_f

In [20]:
model_4 = create_model_4()

# Fit the model
history_4 = model_4.fit(
    train_data,
    epochs = EPOCHS,
    steps_per_epoch = steps_per_epoch,
    validation_data = val_data,
    validation_steps = len(val_data),
    callbacks = [reduce_lr],
)

model_4.save("/kaggle/working/ResNet50.h5")
pd.DataFrame(history_4.history).to_csv("/kaggle/working/History-ResNet50.csv")

94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/128
1233/1233 [==============================] - 116s 90ms/step - loss: 1.9051 - accuracy: 0.8224 - auc: 0.9473 - recall: 0.8181 - precision: 0.8332 - f1_score: 0.8251 - val_loss: 0.9665 - val_accuracy: 0.8625 - val_auc: 0.9664 - val_recall: 0.8592 - val_precision: 0.8697 - val_f1_score: 0.8648 - lr: 0.0100
Epoch 2/128
1233/1233 [==============================] - 132s 107ms/step - loss: 0.3563 - accuracy: 0.9303 - auc: 0.9879 - recall: 0.9266 - precision: 0.9373 - f1_score: 0.9319 - val_loss: 0.4059 - val_accuracy: 0.9138 - val_auc: 0.9854 - val_recall: 0.9106 - val_precision: 0.9216 - val_f1_score: 0.9163 - lr: 0.0100
Epoch 3/128
1233/1233 [==============================] - 108s 88ms/step - loss: 0.2398 - accuracy: 0.9470 - auc: 0.9924 - recall: 0.9440 - precision: 0.9530 - f1_score: 0.9484 - val_loss: 1.0868 - val_accuracy: 0.8688 - val_auc: 0.9645 - val_recall: 0.8660 - val_precision: 0.8726 - val_f1_score: 0.8

In [18]:
model_5 = create_model_5()

# Fit the model
history_5 = model_5.fit(
    train_data,
    epochs = EPOCHS,
    steps_per_epoch = steps_per_epoch,
    validation_data = val_data,
    validation_steps = len(val_data),
    callbacks = [reduce_lr],
)

model_5.save("/kaggle/working/Xception.h5")
pd.DataFrame(history_5.history).to_csv("/kaggle/working/History-Xception.csv")

83683744/83683744 [==============================] - 1s 0us/step
Epoch 1/128
1233/1233 [==============================] - 440s 349ms/step - loss: 0.8551 - accuracy: 0.7294 - auc: 0.9757 - recall: 0.6548 - precision: 0.8941 - f1_score: 0.7479 - val_loss: 0.5675 - val_accuracy: 0.8153 - val_auc: 0.9888 - val_recall: 0.7667 - val_precision: 0.9371 - val_f1_score: 0.8433 - lr: 0.0010
Epoch 2/128
1233/1233 [==============================] - 122s 98ms/step - loss: 0.5222 - accuracy: 0.8303 - auc: 0.9902 - recall: 0.7877 - precision: 0.9403 - f1_score: 0.8569 - val_loss: 0.5152 - val_accuracy: 0.8302 - val_auc: 0.9904 - val_recall: 0.7892 - val_precision: 0.9368 - val_f1_score: 0.8564 - lr: 0.0010
Epoch 3/128
1233/1233 [==============================] - 127s 102ms/step - loss: 0.4422 - accuracy: 0.8550 - auc: 0.9928 - recall: 0.8168 - precision: 0.9535 - f1_score: 0.8795 - val_loss: 0.4743 - val_accuracy: 0.8488 - val_auc: 0.9909 - val_recall: 0.8118 - val_precision: 0.9461 - val_f1_score: 0.

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
model_6 = create_model_6()

# Fit the model
history_6 = model_6.fit(
    train_data,
    epochs = EPOCHS,
    steps_per_epoch = steps_per_epoch,
    validation_data = val_data,
    validation_steps = len(val_data),
    callbacks = [reduce_lr],
)

model_6.save("/kaggle/working/MobileNetV2.h5")
pd.DataFrame(history_6.history).to_csv("/kaggle/working/History-MobileNetV2.csv")

9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/128
1233/1233 [==============================] - 88s 67ms/step - loss: 13.3632 - accuracy: 0.5145 - auc: 0.7904 - recall: 0.5124 - precision: 0.5193 - f1_score: 0.5156 - val_loss: 11.8949 - val_accuracy: 0.6010 - val_auc: 0.8340 - val_recall: 0.5988 - val_precision: 0.6093 - val_f1_score: 0.6038 - lr: 0.0100
Epoch 2/128
1233/1233 [==============================] - 86s 69ms/step - loss: 5.8360 - accuracy: 0.6445 - auc: 0.8627 - recall: 0.6420 - precision: 0.6499 - f1_score: 0.6460 - val_loss: 7.1221 - val_accuracy: 0.6441 - val_auc: 0.8586 - val_recall: 0.6411 - val_precision: 0.6484 - val_f1_score: 0.6448 - lr: 0.0100
Epoch 3/128
1233/1233 [==============================] - 86s 69ms/step - loss: 4.3824 - accuracy: 0.6909 - auc: 0.8864 - recall: 0.6886 - precision: 0.6965 - f1_score: 0.6925 - val_loss: 6.6398 - val_accuracy: 0.5480 - val_auc: 0.8123 - val_recall: 0.5464 - val_precision: 0.5498 - val_f1_score: 0.5478 

# **Results**

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tensorflow.keras.saving import load_model

class_names = os.listdir('/kaggle/input/adverserial-stsd')

# **EfficientNetV2B0**

In [ ]:
metrics = list(history_1.history.keys())

for metric in metrics:
    if metric.startswith('val')==False and metric!='lr':
        plt.figure(figsize=(6, 4))
        plt.plot(history_1.history[metric])
        plt.plot(history_1.history['val_'+metric])
        plt.title('Model '+metric)
        plt.ylabel(metric)
        plt.xlabel('Epoch')
        if metric.endswith('loss'):
            plt.legend(['Train', 'Validation'], loc='upper right')
        else:
            plt.legend(['Train', 'Validation'], loc='lower right')

plt.show()

# **EfficientNetB0**

In [ ]:
metrics = list(history_2.history.keys())

for metric in metrics:
    if metric.startswith('val')==False and metric!='lr':
        plt.figure(figsize=(6, 4))
        plt.plot(history_2.history[metric])
        plt.plot(history_2.history['val_'+metric])
        plt.title('Model '+metric)
        plt.ylabel(metric)
        plt.xlabel('Epoch')
        if metric.endswith('loss'):
            plt.legend(['Train', 'Validation'], loc='upper right')
        else:
            plt.legend(['Train', 'Validation'], loc='lower right')

plt.show()

# **InceptionV3**

In [ ]:
metrics = list(history_3.history.keys())

for metric in metrics:
    if metric.startswith('val')==False and metric!='lr':
        plt.figure(figsize=(6, 4))
        plt.plot(history_3.history[metric])
        plt.plot(history_3.history['val_'+metric])
        plt.title('Model '+metric)
        plt.ylabel(metric)
        plt.xlabel('Epoch')
        if metric.endswith('loss'):
            plt.legend(['Train', 'Validation'], loc='upper right')
        else:
            plt.legend(['Train', 'Validation'], loc='lower right')

plt.show()

# **ResNet50**

In [ ]:
metrics = list(history_4.history.keys())

for metric in metrics:
    if metric.startswith('val')==False and metric!='lr':
        plt.figure(figsize=(6, 4))
        plt.plot(history_4.history[metric])
        plt.plot(history_4.history['val_'+metric])
        plt.title('Model '+metric)
        plt.ylabel(metric)
        plt.xlabel('Epoch')
        if metric.endswith('loss'):
            plt.legend(['Train', 'Validation'], loc='upper right')
        else:
            plt.legend(['Train', 'Validation'], loc='lower right')

plt.show()

# **Xception**

In [ ]:
# metrics = list(history_5.history.keys())

# for metric in metrics:
#     if metric.startswith('val')==False and metric!='lr':
#         plt.figure(figsize=(6, 4))
#         plt.plot(history_5.history[metric])
#         plt.plot(history_5.history['val_'+metric])
#         plt.title('Model '+metric)
#         plt.ylabel(metric)
#         plt.xlabel('Epoch')
#         if metric.endswith('loss'):
#             plt.legend(['Train', 'Validation'], loc='upper right')
#         else:
#             plt.legend(['Train', 'Validation'], loc='lower right')

# plt.show()

# **MobileNetV2**

In [ ]:
metrics = list(history_6.history.keys())

for metric in metrics:
    if metric.startswith('val')==False and metric!='lr':
        plt.figure(figsize=(6, 4))
        plt.plot(history_6.history[metric])
        plt.plot(history_6.history['val_'+metric])
        plt.title('Model '+metric)
        plt.ylabel(metric)
        plt.xlabel('Epoch')
        if metric.endswith('loss'):
            plt.legend(['Train', 'Validation'], loc='upper right')
        else:
            plt.legend(['Train', 'Validation'], loc='lower right')

plt.show()